In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [76]:
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def get_data(pageNo, keyword):  
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
    driver = webdriver.Chrome(options=chrome_options)

    url = f'https://www.amazon.in/s?k={keyword}&page={pageNo}'
    driver.get(url)

    # Wait for a certain element to be present (adjust as needed)
    driver.implicitly_wait(10)

    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')

    alls = []
    unknown_count = 0  # Counter for Unknown-Product
    for d in soup.find_all('div', class_='sg-col-inner'):
        # Check the first class
        name_element = d.find('span', class_='a-size-medium a-color-base a-text-normal')
        
        # If the first class is not found, check the second class
        if not name_element:
            name_element = d.find('span', class_='a-size-base-plus a-color-base a-text-normal')

        # Skip if the name is not available
        if not name_element:
            continue

        # Extract the name
        name = name_element.text.strip()

        # Count Unknown-Product and skip the first 4
        if name == "Unknown-Product":
            unknown_count += 1
            if unknown_count <= 4:
                continue

        rating_element = d.find('span', attrs={'class': 'a-icon-alt'})
        rating = rating_element.text.strip() if rating_element else "0"

        users_rated_element = d.find('span', attrs={'class': 'a-size-base s-underline-text'})
        users_rated_text = users_rated_element.text.strip() if users_rated_element else "0"
        
        # Use a try-except block to handle the conversion safely
        try:
            users_rated = int(re.sub('[^0-9]', '', users_rated_text))
        except ValueError:
            users_rated = 0

        price_element = d.find('span', attrs={'class': 'a-price-whole'})
        price_text = price_element.text.strip() if price_element else "0"

        # Use a try-except block to handle the conversion safely
        try:
            price = int(re.sub('[^0-9]', '', price_text))
        except ValueError:
            price = 0

        all1 = [name, rating, users_rated, price]
        alls.append(all1)

    driver.quit()

    return alls

result = get_data(1, 'keyboard')

In [77]:
keyword = input("What do you want to search for?")
no_pages = 2

In [78]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i,keyword))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Product Name','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [79]:
df=pd.read_csv("amazon_products.csv")

In [80]:
df.head()

,Product Name,Rating,Customers_Rated,Price
0,"Storio Toys Dancing Cactus Talking Toy, Cactus...",3.7 out of 5 stars,2060,299
1,"Storio Toys Dancing Cactus Talking Toy, Cactus...",3.7 out of 5 stars,2060,299
2,"COSMOBABY Swimming Bath Toys for Toddlers 1-3,...",3.8 out of 5 stars,1586,92
3,Storio Toy Octopus Plushie Reversible Soft Toy...,4.2 out of 5 stars,1210,139
4,Storio 3 Slides 24 Patterns Mini Projector Tor...,3.2 out of 5 stars,783,94


In [81]:
df.columns

Index(['Product Name', 'Rating', 'Customers_Rated', 'Price'], dtype='object')

In [82]:

df.dtypes

Product Name       object
Rating             object
Customers_Rated     int64
Price               int64
dtype: object

In [83]:
df.describe()

,Customers_Rated,Price
count,98.000000,98.000000
mean,1952.275510,275.826531
std,2698.763439,179.516936
min,0.000000,53.000000
25%,261.500000,139.000000
50%,786.500000,248.000000
75%,2120.250000,346.500000
max,14122.000000,899.000000


In [89]:
Customers_Rated_mean=df["Customers_Rated"].mean()

In [90]:
Price_mean=df["Price"]>=df["Price"].mean()

In [91]:
df[(df["Customers_Rated"]>=Customers_Rated_mean) & (df["Price"]>=Price_mean)]

,Product Name,Rating,Customers_Rated,Price
0,"Storio Toys Dancing Cactus Talking Toy, Cactus...",3.7 out of 5 stars,2060,299
1,"Storio Toys Dancing Cactus Talking Toy, Cactus...",3.7 out of 5 stars,2060,299
15,Storio Kids Toys LCD Writing Tablet 8.5Inch E-...,3.7 out of 5 stars,14122,139
17,VGRASSP Walkie Talkie Toys for Kids 2 Way Radi...,2.9 out of 5 stars,2094,399
18,GRAPHENE® 6 Slides 48 Patterns Mini Projector ...,3.4 out of 5 stars,2296,188
19,"Funskool Star Links, Multicoloured Interlockin...",4.4 out of 5 stars,9477,179
20,Babique Unicorn Teddy Bear Plush Soft Toy Cute...,4.2 out of 5 stars,7927,159
21,Toy Imagine™ 8.5 inch LCD Writing Tablet Pad f...,3.8 out of 5 stars,4354,87
23,Babique Plush Soft Toy Cute Kids Animal Home D...,4.3 out of 5 stars,2813,159
24,Cable World® Educational Laptop Computer Toy w...,4.2 out of 5 stars,2207,899
